# Provide geometries from CARTO with geostore IDs
This script reads a CARTO table provides each feature with a geostore ID and writes back the table to CARTO.
## Tables to read:
- [River basins - level 3](https://resourcewatch.carto.com/u/wri-rw/tables/wat_068_rw0_watersheds_edit/public?redirected=true) `SELECT * FROM "wri-rw".wat_068_rw0_watersheds_edit WHERE level = 3`

- [Country geometries - coastal](https://resourcewatch.carto.com/u/wri-rw/dataset/gadm36_0) `SELECT * FROM "wri-rw".gadm36_0 WHERE coastal = True`

- [EEZ geometries](https://resourcewatch.carto.com/u/wri-rw/tables/com_011_1_maritime_boundaries_territorial_waters/public?redirected=true): WRI still working out a few discrepancies with the country iso codes used in the two datasets. We want to ensure consistency between the code columns so a country geometry can be matched with its EEZ geometry. We plan to make some edits to the gid_0 and iso_ter1 fields by next week (May31st week), but the underlying geometries should remain the same.

## Methodology
1. Create a geojson from the CARTO table in new row
2. Get ID for geojson from Geostore in new row
3. Publish to CARTO


In [83]:
import os
import re
import json
import pandas as pd
import geopandas as gpd
import cartoframes as cf

# Cartoframes docs --> https://carto.com/developers/cartoframes/reference/

In [ ]:
os.listdir('./')

In [20]:
## env file for gcs upload
env_path = "../.env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value
        
list(env.keys())

['RW_CARTO_KEY', 'RW_CARTO_ACCOUNT']

## Functions

In [44]:
import geojson, json, requests

def shpToGeojson(s):
    """Using a Shapely object, we should build a Geometry object."""
    if s.geom_type in ['Polygon', 'Point', 'MultiPoint','MultiPolygon']:
        atts={'geojson': {'type': 'FeatureCollection',
                        'features': [{'type': 'Feature',
                            'properties': {},
                            'geometry': geojson.Feature(geometry=s, properties={}).get('geometry')
                                    }]}}
        return atts
    else:
        raise ValueError('shape object was not of suitable geometry type')
    
def registerGeostore(geojson):
        """Register valid geojson to the geostore service. Return the geostore id.
        """
        try:
            body = json.loads(json.dumps(geojson))
        except:
            print('Failed to create body')
            return ''
        header= {
                'Content-Type':'application/json'
                }
        url = 'http://api.resourcewatch.org/v1/geostore'
        r = requests.post(url, headers=header, json=body)
        if r.status_code == 200:
            return r.json().get('data', {}).get('id','')
        else:
            print(f'Failed to register geostore. Error {r.status_code}')
            return f'Error {r.status_code}'

## Authentication

In [21]:
creds = cf.auth.Credentials(username=env['RW_CARTO_ACCOUNT'], api_key=env['RW_CARTO_KEY'])

## Processing

### Accessing the tables
#### Countries geometry

In [27]:
countries_df = cf.io.carto.read_carto('SELECT * FROM "wri-rw".gadm36_0 WHERE coastal = True', credentials=creds)
countries_df.head()

,cartodb_id,the_geom,gid_0,name_0,coastal
0,147,"MULTIPOLYGON (((14.41097 35.78847, 14.41097 35...",MLT,Malta,True
1,149,"MULTIPOLYGON (((19.13928 41.99353, 19.13930 41...",MNE,Montenegro,True
2,148,"MULTIPOLYGON (((97.79915 8.83028, 97.79944 8.8...",MMR,Myanmar,True
3,204,"MULTIPOLYGON (((-56.13278 46.78750, -56.13306 ...",SPM,Saint Pierre and Miquelon,True
4,207,"MULTIPOLYGON (((6.52930 0.00403, 6.52930 0.003...",STP,São Tomé and Príncipe,True


#### Watersheds

In [26]:
watersheds_df = cf.io.carto.read_carto('SELECT * FROM "wri-rw".wat_068_rw0_watersheds_edit WHERE level = 3', credentials=creds)
watersheds_df.head()

,cartodb_id,the_geom,hybas_id,next_down,next_sink,main_bas,dist_sink,dist_main,sub_area,up_area,pfaf_id,endo,coast,_order,sort,level
0,7,"MULTIPOLYGON (((35.44861 -23.82500, 35.44252 -...",1030011670,0,1030011670,1030011670,0.0,0.0,245186.1,245186.1,123,0,1,0,7,3
1,8,"MULTIPOLYGON (((29.87083 -25.81250, 29.86998 -...",1030012590,0,1030012590,1030012590,0.0,0.0,412581.2,412581.2,124,0,0,1,8,3
2,9,"MULTIPOLYGON (((26.29028 -33.84583, 26.28531 -...",1030012600,0,1030012600,1030012600,0.0,0.0,403562.0,403562.0,125,0,1,0,9,3
3,10,"MULTIPOLYGON (((19.41944 -34.69167, 19.40928 -...",1030015030,0,1030015030,1030015030,0.0,0.0,111328.1,111328.1,126,0,1,0,10,3
4,11,"MULTIPOLYGON (((22.97500 -31.57917, 22.97379 -...",1030015850,0,1030015850,1030015850,0.0,0.0,977324.0,977324.0,127,0,0,1,11,3


#### EEZ 

In [28]:
eez_table = 'com_011_1_maritime_boundaries_territorial_waters'
eez_df = cf.io.carto.read_carto(eez_table, credentials=creds)

eez_df.head()

,cartodb_id,the_geom,mrgid,geoname,pol_type,mrgid_ter1,territory1,mrgid_sov1,sovereign1,iso_ter1,x_1,y_1,mrgid_eez,area_km2
0,160,"MULTIPOLYGON (((-13.70488 9.52191, -13.70493 9...",49186,Guinean 12 NM,12NM,2122,Guinea,2122,Guinea,GIN,-14.185269,9.947250,8472,9306.0
1,161,"MULTIPOLYGON (((-3.16821 4.89856, -3.17263 4.8...",49187,Ivory Coast 12 NM,12NM,2161,Ivory Coast,2161,Ivory Coast,CIV,-5.271691,4.895276,8473,12375.0
2,162,"MULTIPOLYGON (((8.45250 4.61667, 8.44333 4.600...",49188,Nigerian 12 NM,12NM,2253,Nigeria,2253,Nigeria,NGA,5.607847,5.174516,8474,20404.0
3,166,"MULTIPOLYGON (((12.39538 -6.32436, 12.39728 -6...",49192,Angolan 12 NM,12NM,2150,Angola,2150,Angola,AGO,12.684313,-11.326831,8478,34318.0
4,167,"MULTIPOLYGON (((39.72996 -4.68341, 39.77037 -4...",49193,Tanzanian 12 NM,12NM,2205,Tanzania,2205,Tanzania,TZA,39.926449,-7.641209,8479,16008.0


### Creating geojson and getting geostore id

#### Countries geometries

In [62]:
countries_g_id_dict = {}
countries_geojson_dict = {}
for index, row in countries_df.iterrows():
    g = shpToGeojson(row.the_geom)
    g_id = registerGeostore(g)
    if re.search('^Error ', g_id)!=None: #if there is an error with geostore
        countries_geojson_dict[row.cartodb_id] = g
    else: 
        countries_g_id_dict[row.cartodb_id] = g_id

Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 413
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 413
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geostore. Error 500
Failed to register geosto

In [61]:
print(len(countries_g_id_dict))
print(len(countries_geojson_dict))
len(countries_g_id_dict)+len(countries_geojson_dict)

208

In [ ]:
countries_df.shape

In [86]:
# save dicts locally
with open('countries_g_id_dict.json', 'w') as fp:
    json.dump(countries_g_id_dict, fp)
with open('countries_geojson_dict.json', 'w') as fp:
    json.dump(countries_geojson_dict, fp)

In [34]:
countries_df.to_csv(f'./countries_df_v20210528.csv')

#### Watersheds

In [46]:
watershed_g_id_dict = {}
watershed_geojson_dict = {}
for index, row in watersheds_df.iterrows():
    g = shpToGeojson(row.the_geom)
    g_id = registerGeostore(g)
    if re.search('^Error ', g_id)!=None: #if there is an error with geostore
        watershed_geojson_dict[row.cartodb_id] = g
    else: 
        watershed_g_id_dict[row.cartodb_id] = g_id

In [59]:
print(len(watershed_g_id_dict))
print(len(watershed_geojson_dict))
len(watershed_g_id_dict)+len(watershed_geojson_dict)

292
0


292

In [60]:
watersheds_df.shape

(292, 16)

In [85]:
# save dicts locally
with open('watershed_g_id_dict.json', 'w') as fp:
    json.dump(watershed_g_id_dict, fp)


In [81]:
# transform dictionary to table with two columns and merge
watershed_id_df = pd.DataFrame.from_dict(watershed_g_id_dict, orient = 'index', columns = ["geo_id"])
watershed_id_df.reset_index(inplace=True)
watershed_id_df = watershed_id_df.rename(columns = {'index':'cartodb_id'})
watershed_id_df.merge(watersheds_df, left_on='cartodb_id', right_on='cartodb_id')
watershed_id_df.head()

,cartodb_id,geo_id
0,7,5cc9febfda14763c2c4cfdea65eb3b18
1,8,e03ae30a0b3fa9d3bef7b68ec04f6977
2,9,ffd4ce51f1407a2bb64a6d719dc00106
3,10,4291c2aa881be2d025582cba57e9ce42
4,11,d1bd595eaa7ba2dbc65e09a4eab103f9


In [82]:
#save locally
watershed_id_df.to_csv(f'./watersheds_df_id_v20210528.csv')

#### EEZ

In [50]:
eez_g_id_dict = {}
eez_geojson_dict = {}
for index, row in eez_df.iterrows():
    g = shpToGeojson(row.the_geom)
    g_id = registerGeostore(g)
    if re.search('^Error ', g_id)!=None: #if there is an error with geostore
        eez_geojson_dict[row.cartodb_id] = g
    else: 
        eez_g_id_dict[row.cartodb_id] = g_id

Failed to register geostore. Error 500
Failed to register geostore. Error 504
Failed to register geostore. Error 500
Failed to register geostore. Error 500


In [57]:
eez_df.shape

(233, 14)

In [58]:
print(len(eez_g_id_dict))
print(len(eez_geojson_dict))
len(eez_g_id_dict)+len(eez_geojson_dict)

229
4


233

In [84]:
# save dicts locally
with open('eez_g_id_dict.json', 'w') as fp:
    json.dump(eez_g_id_dict, fp)
with open('eez_geojson_dict.json', 'w') as fp:
    json.dump(eez_geojson_dict, fp)


In [23]:
## save locally
eez_df.to_csv(f'./{eez_table}_v20210528.csv')

## check failed registration, `geostore_id  == ''`
- for countries
- for eez

### Countries

In [63]:
## check failed registration, geostore_id  == ''
## s.is_valid, s.make_valid / buffer(0)

### EEZ

In [64]:
## check failed registration, geostore_id  == ''
## s.is_valid, s.make_valid / buffer(0)

## write carto table

### Watersheds

In [ ]:
# cf.io.carto.to_carto(df, <tablename>, if_exists='replace', credentials=creds)
# cf.update_privacy_table(<tablename>, privacy='public', credentials=creds)

#### Countries

In [ ]:
# cf.io.carto.to_carto(df, <tablename>, if_exists='replace', credentials=creds)
# cf.update_privacy_table(<tablename>, privacy='public', credentials=creds)

#### EEZ

In [ ]:
# cf.io.carto.to_carto(df, <tablename>, if_exists='replace', credentials=creds)
# cf.update_privacy_table(<tablename>, privacy='public', credentials=creds)